In [108]:
import pickle
import numpy as np
import utm
from simplekml import Kml, AltitudeMode, Color
from tqdm import tqdm

In [151]:
def utm_to_wgs84(x_coords, y_coords, zone_number=32, zone_letter='N'):
    x_grid, y_grid = np.meshgrid(x_coords, y_coords)
    lat_grid = np.zeros_like(x_grid)
    lon_grid = np.zeros_like(x_grid)

    for i in range(x_grid.shape[0]):
        for j in range(x_grid.shape[1]):
            lat, lon = utm.to_latlon(x_grid[i, j], y_grid[i, j], zone_number, zone_letter)
            lat_grid[i, j] = lat
            lon_grid[i, j] = lon

    lat_coords = lat_grid.mean(axis=1)
    lon_coords = lon_grid.mean(axis=0)

    return lon_coords, lat_coords

In [140]:
with open('xyz_pickles/x_results.pkl','rb') as f:
    x_results = pickle.load(f)

with open('xyz_pickles/y_results.pkl','rb') as f:
    y_results = pickle.load(f)

with open('xyz_pickles/z_results.pkl','rb') as f:
    z_results = pickle.load(f)

In [141]:
x_array = np.array([])

for i in range(len(x_results)):
  x_array = np.concatenate((x_array, x_results[i][0]))

y_array = np.array([])

for j in range(len(y_results)):
  y_array = np.concatenate((y_array, y_results[0][j]))

In [142]:
lst = z_results

row=len(lst)
col=len(lst[0])

for j in range(0, row):
  for i in range(0, col):
    if i==0:
      z_array_row = z_results[i][j]
    else:
      z_array_row = np.hstack((z_array_row, z_results[i][j]))
  
  if j==0:
    z_array = z_array_row
  else:
    z_array = np.vstack((z_array, z_array_row))

In [143]:
z_array = z_array + 300

In [144]:
n = 2

x_array = x_array[::n]
y_array = y_array[::n]
z_array = z_array[::n, ::n]

In [147]:
def create_kml_surface(lon_coords, lat_coords, z_coords, kml_filename='surface.kml'):
    kml = Kml()
    altitude_mode = AltitudeMode.absolute

    for i in range(len(lat_coords) - 1):
        for j in range(len(lon_coords) - 1):
            pol = kml.newpolygon(outerboundaryis=[
                (lon_coords[j], lat_coords[i], z_coords[i, j]),
                (lon_coords[j+1], lat_coords[i], z_coords[i, j+1]),
                (lon_coords[j+1], lat_coords[i+1], z_coords[i+1, j+1]),
                (lon_coords[j], lat_coords[i+1], z_coords[i+1, j]),
                (lon_coords[j], lat_coords[i], z_coords[i, j])
            ])
            pol.altitudemode = altitude_mode
            pol.extrude = 0  # Disable extrusion (remove side walls)
            pol.polystyle.outline = 0  # Disable outline
            pol.polystyle.color = Color.changealphaint(220, Color.red)  # Set color to red with 50% transparency (128)

    kml.save(kml_filename)

In [152]:
lon_array, lat_array = utm_to_wgs84(x_array, y_array)

In [156]:
create_kml_surface(lon_array, lat_array, z_array, 'Obstacles_surface.kml')